<a href="https://colab.research.google.com/github/Bur4ak/Bur4ak.github.io/blob/main/DialogGenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# HÜCRE 1: Gerekli Kütüphaneleri Yükleme ve Güncelleme
# Bu hücreyi sadece bir kez çalıştırın.
# ==============================================================================

print("HÜCRE 1: Kütüphaneler kaldırılıyor ve güncel sürümler yükleniyor...")

# Mevcut olası uyumsuz sürümleri tamamen kaldırmak için agresif uninstall
# (Hata vermesi normaldir, çünkü bazıları yüklü olmayabilir)
!pip uninstall -y transformers datasets accelerate torch torchvision torchaudio bitsandbytes peft trl

# PyTorch, Torchvision ve Torchaudio'yu Colab'in T4 GPU'ları için optimize edilmiş CUDA 12.1 sürümüyle yükleyelim.
print("\nPyTorch, Torchvision, Torchaudio (CUDA 12.1 uyumlu) yükleniyor...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Transformers kütüphanesini belirli bir stabil sürüme sabitleyelim
print("\nTransformers kütüphanesi (belirli sürüm) yükleniyor...")
!pip install transformers==4.51.3 # En son stabil versiyonlardan biri

# Diğer gerekli kütüphaneleri yükleyelim/güncelleyelim
print("\nDiğer yardımcı kütüphaneler yükleniyor/güncelleniyor...")
!pip install datasets accelerate pandas bitsandbytes peft trl

print("\nHÜCRE 1 Tamamlandı: Gerekli kütüphaneler yüklendi ve güncellendi.")

# Yüklü ana kütüphane sürümlerini kontrol edelim (Sadece bilgi amaçlı)
print("\nYüklü ana kütüphane sürümleri:")
!pip show torch
!pip show torchvision
!pip show transformers
!pip show accelerate

# ==============================================================================
# ÖNEMLİ ADIM:
# BU HÜCREYİ ÇALIŞTIRDIKTAN SONRA MUTLAKA YAPIN:
# 1. Colab menüsünden "Çalışma Zamanı (Runtime)" -> "Çalışma zamanını yeniden başlat (Restart runtime)" seçeneğini seçin.
# 2. Çalışma zamanı yeniden başladıktan sonra, not defterinizdeki TÜM SONRAKİ HÜCRELERİ BAŞTAN BAŞLAYARAK SIRA İLE çalıştırın.
# ==============================================================================

HÜCRE 1: Kütüphaneler kaldırılıyor ve güncel sürümler yükleniyor...
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2

PyTorch, Torchvision, Torchaudio (CUDA 12.1 uyumlu) yükleniyor...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ==============================================================================
# HÜCRE 2: Kütüphaneleri İçeri Aktarma
# Bu hücreyi, HÜCRE 1'i çalıştırıp Colab'i yeniden başlattıktan sonra çalıştırın.
# ==============================================================================
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
import torch # GPU kullanılabilirliğini kontrol etmek için

print("\nHÜCRE 2 Tamamlandı: Gerekli kütüphaneler başarıyla içeri aktarıldı.")

# GPU'nun kullanılabilir olduğunu kontrol edelim
if torch.cuda.is_available():
    print(f"GPU kullanılabilir: {torch.cuda.get_device_name(0)}")
else:
    print("GPU kullanılamıyor. Lütfen çalışma zamanı türünüzü GPU olarak ayarladığınızdan emin olun.")


HÜCRE 2 Tamamlandı: Gerekli kütüphaneler başarıyla içeri aktarıldı.
GPU kullanılabilir: Tesla T4


In [ ]:
# ==============================================================================
# HÜCRE 3: Veri Hazırlığı (`dialogues.csv` Dosyasını İşleme)
# dialogues.csv dosyanızın Colab ortamına yüklendiğinden emin olun.
# ==============================================================================
try:
    df = pd.read_csv('dialogues.csv')
    print("\nHÜCRE 3: dialogues.csv dosyası başarıyla yüklendi.")
    print("Dosya sütunları:", df.columns.tolist())

    df['text'] = "NPC: " + df['dialogue'].astype(str) + "\nPlayer: " + df['player_response'].astype(str)
    print("Metin verileri birleştirildi. İlk 3 örnek:")
    print(df['text'].head(3))

    dataset = Dataset.from_pandas(df)
    train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = train_test_split['train']
    eval_dataset = train_test_split['test']

    print(f"\nEğitim seti boyutu: {len(train_dataset)}")
    print(f"Doğrulama seti boyutu: {len(eval_dataset)}")

except FileNotFoundError:
    print("Hata: dialogues.csv dosyası bulunamadı. Lütfen dosyayı Colab ortamına yüklediğinizden emin olun.")
    raise SystemExit("Dosya hatası, program sonlandırılıyor.")


HÜCRE 3: dialogues.csv dosyası başarıyla yüklendi.
Dosya sütunları: ['npc', 'dialogue', 'player_response']
Metin verileri birleştirildi. İlk 3 örnek:
0    NPC: Hi, how are you doing? I'm getting ready ...
1    NPC: Hi, how are you doing? I'm getting ready ...
2    NPC: I am! For my hobby I like to do canning o...
Name: text, dtype: object

Eğitim seti boyutu: 381785
Doğrulama seti boyutu: 42421


In [ ]:
# ==============================================================================
# HÜCRE 4: Model ve Tokenizer Seçimi ve Yüklenmesi
# ==============================================================================
model_name = "ytu-ce-cosmos/turkish-gpt2" # Veya "cenkersisman/gpt2-turkish-50m" gibi daha küçük bir alternatif

print(f"\nHÜCRE 4: '{model_name}' model ve tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

print("Model ve tokenizer başarıyla yüklendi.")


HÜCRE 4: 'ytu-ce-cosmos/turkish-gpt2' model ve tokenizer yükleniyor...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Model ve tokenizer başarıyla yüklendi.


In [ ]:
# ==============================================================================
# HÜCRE 5: Veriyi Tokenleştirme
# ==============================================================================
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

print("\nHÜCRE 5: Veri tokenleştiriliyor...")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=df.columns.tolist())
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=df.columns.tolist())

print("Veri tokenleştirme tamamlandı.")
print(f"Tokenleştirilmiş eğitim seti ilk örnek (input_ids): {tokenized_train_dataset[0]['input_ids']}")


HÜCRE 5: Veri tokenleştiriliyor...


Map:   0%|          | 0/381785 [00:00<?, ? examples/s]

Map:   0%|          | 0/42421 [00:00<?, ? examples/s]

Veri tokenleştirme tamamlandı.
Tokenleştirilmiş eğitim seti ilk örnek (input_ids): [46, 22596, 26, 11434, 1264, 6240, 29413, 47797, 657, 1208, 556, 2529, 869, 473, 297, 6762, 1589, 3707, 16144, 14411, 83, 2648, 473, 556, 21933, 22561, 14, 199, 24612, 258, 26, 809, 29171, 11655, 537, 1627, 9320, 31]


In [ ]:
# ==============================================================================
# HÜCRE 6: Fine-Tuning (Hugging Face Trainer API ile)
# ==============================================================================
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("\nHÜCRE 6: Fine-tuning işlemi başlatılıyor...")
trainer.train()

model_save_path = "./my_finetuned_dialog_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"\nHÜCRE 6 Tamamlandı: Model ve tokenizer başarıyla '{model_save_path}' dizinine kaydedildi.")

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# ==============================================================================
# HÜCRE 6: Fine-Tuning (Hugging Face Trainer API ile) - GEÇİCİ ÇÖZÜM UYGULANDI
# ==============================================================================
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Eğitim argümanları
# evaluation_strategy ve save_strategy argümanları hata verdiği için kaldırıldı.
# Bu durumda model eğitimi sırasında otomatik değerlendirme ve kaydetme yapılmayacaktır.
# Eğitim sonunda model yine de kaydedilecektir.
training_args = TrainingArguments(
    output_dir="./results",          # Eğitimin çıktı dosyalarının kaydedileceği dizin
    num_train_epochs=1,              # Eğitim döngüsü sayısı
    per_device_train_batch_size=16,   # Her GPU/CPU cihazı başına eğitim batch boyutu
    per_device_eval_batch_size=4,    # Her GPU/CPU cihazı başına değerlendirme batch boyutu
    warmup_steps=500,                # Öğrenme oranının başlangıçta yavaşça arttırıldığı adım sayısı
    weight_decay=0.01,               # Overfitting'i azaltmak için uygulanan ağırlık azaltma
    logging_dir="./logs",            # Log dosyalarının kaydedileceği dizin
    logging_steps=50,                # Kaç adımda bir eğitim loglarının kaydedileceği
    # evaluation_strategy="epoch",   # Bu satır hata verdiği için kapatıldı/kaldırıldı
    # save_strategy="epoch",         # Bu satır hata verdiği için kapatıldı/kaldırıldı
    load_best_model_at_end=False,    # evaluation_strategy olmadığı için bu False olmalı
    # metric_for_best_model="eval_loss", # evaluation_strategy olmadığı için bu kaldırıldı
    fp16=torch.cuda.is_available(),  # Eğer GPU varsa 16-bit kayan nokta hassasiyeti kullanarak eğitimi hızlandır
    gradient_accumulation_steps=2,   # Eğer GPU belleği yetersizse, daha büyük efektif batch boyutları için gradyanları biriktirme
    report_to="none"
)

# Trainer nesnesini oluşturma
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset, # Doğrulama seti hala kullanılacak ama değerlendirme otomatik yapılmayacak
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("\nHÜCRE 6: Fine-tuning işlemi başlatılıyor...")
trainer.train()

# Eğitilmiş modeli ve tokenizer'ı kaydetme
model_save_path = "./my_finetuned_dialog_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"\nHÜCRE 6 Tamamlandı: Model ve tokenizer başarıyla '{model_save_path}' dizinine kaydedildi.")

<ipython-input-10-e92eab01853e>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



HÜCRE 6: Fine-tuning işlemi başlatılıyor...


Step,Training Loss
50,2.252100
100,2.238900
150,2.160400
200,2.113700
250,2.040300
300,2.019200
350,2.322100
400,2.269800
450,2.271500
500,2.228200



HÜCRE 6 Tamamlandı: Model ve tokenizer başarıyla './my_finetuned_dialog_model' dizinine kaydedildi.


In [ ]:
# ==============================================================================
# HÜCRE 7: Eğitilmiş Modeli Kullanma (İnteraktif Diyalog Testi)
# Bu bölüm, fine-tuning işlemi tamamlandıktan sonra modelinizi hemen test etmenizi sağlar.
# ==============================================================================
from transformers import pipeline

print(f"\nHÜCRE 7: '{model_save_path}' dizininden model yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForCausalLM.from_pretrained(model_save_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dialog_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_player_response(npc_dialogue_text, max_length=100, temperature=0.7, top_k=50, top_p=0.95):
    prompt = f"NPC: {npc_dialogue_text}\nPlayer:"

    generated_output = dialog_pipeline(
        prompt,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )

    full_text = generated_output[0]['generated_text']

    if "Player:" in full_text:
        response_start_index = full_text.find("Player:") + len("Player:")
        player_response = full_text[response_start_index:].strip()

        if "NPC:" in player_response:
            player_response = player_response.split("NPC:")[0].strip()

        if not player_response or len(player_response) < 5:
            return "Üzgünüm, anlayamadım. Lütfen tekrar deneyin."
        return player_response

    return "Yanıt oluşturulamadı."

print("\n--- HÜCRE 7 Tamamlandı: NPC Diyalog Botu (Test Başladı) ---")
print("Çıkmak için klavyeden 'çıkış' yazıp Enter'a basın.")
print("-------------------------------------------------------")
while True:
    npc_input = input("NPC: ")
    if npc_input.lower() == "çıkış":
        print("Diyalog sonlandırıldı.")
        break

    player_output = generate_player_response(npc_input)
    print(f"Oyuncu: {player_output}")


HÜCRE 7: './my_finetuned_dialog_model' dizininden model yükleniyor...


Device set to use cuda:0



--- HÜCRE 7 Tamamlandı: NPC Diyalog Botu (Test Başladı) ---
Çıkmak için klavyeden 'çıkış' yazıp Enter'a basın.
-------------------------------------------------------
NPC: Merhaba


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Oyuncu: I'm a woman, I'm not a woman. I'm a little old. You?
Player: I'm a woman. I'm a little old too. I'm also a woman. What about you? I'm a woman. You? I'm a little old!
Player: I'm an older man. You? I'm a woman. I'm older than
NPC: How old are you ?
Oyuncu: I'm 18. how old are you? I'm 30. you? How old are you? I'm older. You? Haha. I'm 18. you? I'm older. Lol. you?
Player: I'm older. Just like chocolate chip cookies. Whats your favorite? Mine is a pepsi. I'm a pepsi fan. Pepsi fan. Y
NPC: Hello how are you
Oyuncu: I'm doing good. Just got back from a run. How are you? Do you run? I do. you? I run.
Player: I run a lot too. I love to run. I'm great at it! What do you do? I run. I like to ride my bike. Not to work. Do you? I do. you? P.
Play
NPC: what is your hobbys ?
Oyuncu: I love to cook, do you have any? I like to sleep. What about you? You? Do you have any pets? I've a lizard. I've a lizard.
Player: I do not have any pets. I've a pug. I do not like dogs. I love to. You? What d